## Task 1: Get Info Box, store it to python dictionary

###### Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

##### Load the webpage

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# Convert to beautiful soup object
soup = bs(r.content)

# Print out HTML
contents = soup.prettify()
# print(contents)

In [3]:
info_box = soup.find(class_="infobox vevent")
# print(info_box.prettify())

In [4]:
info_rows = info_box.find_all("tr")

# for row in info_rows:
#     print(row.prettify())

In [5]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

movie_info = {}
for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip=True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value
        
# movie_info

## Task 2: Get info box for all movies

In [6]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to a beautiful soup object
soup = bs(r.content)

contents = soup.prettify()
# contents

In [7]:
movies = soup.select('.wikitable.sortable i') # i means only italized items
# movies

In [8]:
# print(movies[0])
# print("")
# print(movies[0].a['href']) 

In [9]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

def get_info_box(url):
    r = requests.get(url)
\
    soup = bs(r.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all('tr')
    
    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        elif index == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True)
            content_value = get_content_value(row.find("td"))
            movie_info[content_key] = content_value
    return movie_info

In [10]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to a beautiful soup object
soup = bs(r.content)
movies = soup.select('.wikitable.sortable i a')
# movies

In [11]:
base_path = "https://en.wikipedia.org/"
movie_info_list = []

for index, movie in enumerate(movies):
    try:
        relative_path = movie['href']
        title = movie['title']
        full_path = base_path + relative_path
        
        movie_info_list.append(get_info_box(full_path))
#         print(relative_path)
#         print(title)
#         print()
    except Exception as e:
        print(movie.get_text())
        print(e)
# movie_info_list

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
One Little Indian
'NoneType' object has no attribute 'get_text'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
Spirited Away
'NoneType' object has no attribute 'get_text'
Howl's Moving Castle
'NoneType' object has no attribute 'get_text'
The Nightmare Before Christmas 3D
'NoneType' object has no attribute 'get_text'
The Secret of the Magic Gourd
'NoneType' object has no attribute 'get_text'
Ponyo
'NoneType' object has no attribute 'get_text'
Tales from Earthsea
'NoneType' object has no attribute 'get_text'
The Secret World of Arrietty
'NoneType' object has no attribute 'get_text'
Khoobsurat
'NoneType' object has no attribute 'get_text'
Tini: The Movie
'NoneType' object has no attribute 'get_text'
Born in China
'NoneType' object has no attribute 'get_text'
Night at the Museum: Kahmunrah Rises Again
'NoneType' object has no attribute 'find'


In [15]:
movie_info_list[0]

{'title': 'Academy Award Review of Walt Disney Cartoons',
 'Production company': 'Walt Disney Productions',
 'Release date': ['May 19, 1937 ( 1937-05-19 )'],
 'Running time': '41 minutes (74 minutes 1966 release)',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$45.472'}

In [13]:
len(movie_info_list)

435

#### Save/Reload Movie Data

In [16]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:
import json

def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)